In [1]:
# If you have not done so already: pip install sklearn-crfsuite
import sklearn_crfsuite
import pandas as pd
import glob
from csv import DictReader

In [2]:
filename = './../../Data/parc_features/parc_dev_features.tsv'

In [8]:
def extract_features_and_labels(list_of_dicts):
    feature_dicts = []
    label_list = []
    for token_dict in list_of_dicts:
        label = token_dict.pop('content_label_gold')
        feature_dicts.append(token_dict)
        label_list.append(label)
        
    return [feature_dicts], [label_list]

def train_crf(X_train, y_train):  
    crf = sklearn_crfsuite.CRF(  
        algorithm='lbfgs',  
        c1=0.1,  
        c2=0.1,  
        max_iterations=100,  
        all_possible_transitions=True
    )  
    return crf.fit(X_train, y_train)

In [9]:
with open(filename, 'r') as infile:
    dict_reader = DictReader(infile, delimiter='\t')
    train_data = list(dict_reader)
    
#train_data = train_data[:10]
print(train_data[0])

OrderedDict([('', '0'), ('POS', 'DT'), ('dependency_head', '2'), ('dependency_label', 'det'), ('doc_token_number', '1'), ('lemma', 'the'), ('ne_info', 'O'), ('sentence_number', '1'), ('sentence_token_number', '1'), ('token', 'The'), ('cue_label', '0'), ('attribution', '_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _'), ('token_-1', '.'), ('token_-2', '.'), ('token_-3', '.'), ('token_-4', '.'), ('token_-5', '.'), ('token_+1', 'economy'), ('token_+2', "'s"), ('token_+3', 'temperature'), ('token_+4', 'will'), ('token_+5', 'be'), ('lemma_-1', '.'), ('lemma_-2', '.'), ('lemma_-3', '.'), ('lemma_-4', '.'), ('lemma_-5', '.'), ('lemma_+1', 'economy'), ('lemma_+2', "'s"), ('lemma_+3', 'temperature'), ('lemma_+4', 'will'), ('lemma_+5', 'be'), ('POS_-1', '.'), ('POS_-2', '.'), ('POS_-3', '.'), ('POS_-4', '.'), ('POS_-5', '.'), ('POS_+1', 'NN'), ('POS_+2', 'POS'), ('POS_+3', 'NN'), ('POS_+4', 'MD'), ('POS_+5', 'VB'), ('bigram_prev_token', 'The .'), ('bigram_prev_lemma', 'the .'), ('bigram_prev

In [13]:
X, y = extract_features_and_labels(train_data)

In [18]:
X_train = [X[0][:21000]]
X_test = [X[0][21000:]]
y_train = [y[0][:21000]]
y_test = [y[0][21000:]]

In [19]:
print(len(X_train))
print(len(X_train[0]))
print()
print('y_train')
print(len(y_train))
print(len(y_train[0]))

1
21000

y_train
1
21000


In [25]:
def train_crf(X_train, y_train):
    crf = sklearn_crfsuite.CRF(  
            algorithm='lbfgs',  
            c1=0.1,  
            c2=0.1,  
            max_iterations=100,  
            all_possible_transitions=True  
        ) 
    crf.fit(X_train, y_train)
    return crf

def test_crf_prediction(crf, X_test, y_test):  
    #labels = show_crf_label(crf)  
    y_pred = crf.predict(X_test)  
    return y_pred
    

In [26]:
crf = train_crf(X_train, y_train)

In [28]:
y_pred = test_crf_prediction(crf, X_test, y_test)

In [29]:
y_pred

[['I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'B',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',